# Qiskit Coding Tasks: Quantum Teleportation

This notebook contains coding exercises to build the Quantum Teleportation protocol from the ground up.

**Setup:**
You will need `qiskit` and `numpy`. Install them if needed:
`pip install qiskit qiskit-aer numpy`

## Setup: Imports for all tasks

In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile, ClassicalRegister, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from numpy import pi

# Set up the simulator
simulator = AerSimulator()

--- 

## Task 1: The Initial 3-Qubit State

### 1. Engage (Why?)

*Multiple Means of Engagement (Recruiting Interest)*

The teleportation protocol requires 3 qubits: Alice's "cat" qubit ($q_C$), her half of the Bell pair ($q_A$), and Bob's half ($q_B$). The total system state is the tensor product of these parts: $|\Psi_{total}
\rangle = |\psi
\rangle_C \otimes |\Phi^+
\rangle_{AB}$. Let's build this state vector manually to prove we understand the setup.

### 2. Explore (Play)

*Multiple Means of Action & Expression (Fluency)*

Let's pick a simple, non-trivial state for our "cat" qubit: $|\psi
\rangle_C = |+
\rangle = \frac{1}{\sqrt{2}}(|0
\rangle + |1
\rangle)$. Let's also create the Bell state $|\Phi^+
\rangle_{AB}$. We can then use `Statevector.tensor()` to combine them.

In [3]:
# Create |+>
sv_cargo = Statevector([1/np.sqrt(2), 1/np.sqrt(2)])

# Create |Phi+> = 1/sqrt(2) * [1, 0, 0, 1]
sv_bell = Statevector([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)])

# Combine them: |+> tensor |Phi+>
sv_total = sv_cargo.tensor(sv_bell)

print("Initial 3-Qubit State Vector:")
print(sv_total)

Initial 3-Qubit State Vector:
Statevector([0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j,
             0.5+0.j],
            dims=(2, 2, 2))


### 3. Explain (The Concept)

*Multiple Means of Representation (Clarify Notation)*

This 8-dimensional vector is our starting point, $|\Psi_{total}
\rangle$. The 8 entries correspond to the amplitudes for $|000
\rangle, |001
\rangle, |010
\rangle, ..., |111
\rangle$. 
Manually calculating tensor products is tedious. A better way to get this state is to build a quantum circuit that *produces* it.

### 4. Elaborate (Apply)

*Multiple Means of Action & Expression (Higher-level skills)*

Let's create the initial state $|\Psi_{total}
\rangle = |+
\rangle_C \otimes |\Phi^+
\rangle_{AB}$ using a circuit.

- We need 3 qubits: $q_2$ (Cat), $q_1$ (Alice's), $q_0$ (Bob's).
- To create $|+
\rangle$ on $q_2$, what gate do you apply?
- To create $|\Phi^+
\rangle$ on $q_1, q_0$, what two gates do you apply?
- Note that the notation in qiskit is |$q_2 q_1 q_0\rangle$, we will follow qiskit notation but align with the order of the qubit that you've learned from the notes

In [7]:
# --- Your Code Here --- 
qc_init = QuantumCircuit(3)

# 1. Create |+> on q2 (Cat)
qc_init.h(2)

# 2. Create |Phi+> on q1 (Alice) and q0 (Bob)
qc_init.h(1)
qc_init.cx(1, 0)

print("Circuit to create initial state |+> ⊗ |Φ+>:")
print(qc_init.draw())

# --- End Your Code ---

# Let's verify this circuit gives the same statevector
sv_from_circuit = Statevector(qc_init)

print("\nStatevector from circuit:")
print(sv_from_circuit)

# Check if they are the same (ignoring global phase)
print(f"\nDo the vectors match? {sv_total.equiv(sv_from_circuit)}")

Circuit to create initial state |+> ⊗ |Φ+>:
          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     ├───┤     
q_2: ┤ H ├─────
     └───┘     

Statevector from circuit:
Statevector([0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j,
             0.5+0.j],
            dims=(2, 2, 2))

Do the vectors match? True


Compare with general states, is it the same? What is $\alpha$ and $\beta$
$$ 
|\Psi_{total}
\rangle = \frac{1}{\sqrt{2}} \left[ \alpha(|000
\rangle + |011
\rangle) + \beta(|100
\rangle + |111
\rangle) 
\right] 
$$ 

### 5. Evaluate (Challenge)

*Multiple Means of Engagement (Mastery)*

This circuit is our "initial state preparation." Now we can build the rest of the protocol on top of it. 

**Challenge:** 

How would you change the circuit above if the cargo state to be sent was $|\psi
\rangle_C = |-
\rangle$ instead of $|+
\rangle$? What if it was $|\psi
\rangle_C = |1
\rangle$?

In [ ]:
# --- Your Code Here ---
# Hint: Qubit to be teleported is does not affect the circuit structure, only the initial state preparation.
# --- End Your Code ---

--- 

## Task 2: Alice's Operations (Bell Basis Measurement)

### 1. Engage (Why?)

*Multiple Means of Engagement (Relevance)*

Alice needs to perform a "Bell Basis Measurement" on *her* two qubits ($q_2$ and $q_1$). As your notes show, this involves applying a $CNOT$ and a $Hadamard$ gate. This is the part of the circuit that *uses* the entanglement.

### 2. Explore (Play)

*Multiple Means of Action & Expression (Fluency)*

Let's add these gates to our circuit. The notes show the order is:
1.  $CNOT$ from $q_C 	o q_A$ (that's `qc.cx(2, 1)`)
2.  $Hadamard$ on $q_C$ (that's `qc.h(2)`)

We also need two classical bits for Alice to store her measurement results.

In [ ]:
# Create registers: 3 qubits, 2 classical bits
qr = QuantumRegister(3, name="q")
cr_alice = ClassicalRegister(2, name="c_alice")
qc_alice_ops = QuantumCircuit(qr, cr_alice)

# --- Start with the initial state from Task 1 --- 
# (We'll just re-build it here for clarity)
# Let's teleport |+>
qc_alice_ops.h(qr[2]) # Prepare |+> on q2
qc_alice_ops.h(qr[1]) # Prepare |Phi+> on q1, q0
qc_alice_ops.cx(qr[1], qr[0])
qc_alice_ops.barrier()

# --- Add Alice's Bell Basis Measurement gates --- 
qc_alice_ops.cx(qr[2], qr[1]) # CNOT(C, A)
qc_alice_ops.h(qr[2])      # H(C)
qc_alice_ops.barrier()

# Check the statevector before measurement
sv_before_measure = Statevector(qc_alice_ops)
print("\nStatevector before Alice's measurement:")
print(sv_before_measure)

# --- Add Alice's measurement --- 
qc_alice_ops.measure(qr[2], cr_alice[0]) # Measure q2 into c0
qc_alice_ops.measure(qr[1], cr_alice[1]) # Measure q1 into c1

print("Circuit with Alice's operations:")
print(qc_alice_ops.draw())


Statevector before Alice's measurement:
Statevector([ 0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,
              0.35355339+0.j,  0.35355339+0.j, -0.35355339+0.j,
             -0.35355339+0.j,  0.35355339+0.j],
            dims=(2, 2, 2))
Circuit with Alice's operations:
                ┌───┐ ░            ░       
      q_0: ─────┤ X ├─░────────────░───────
           ┌───┐└─┬─┘ ░ ┌───┐      ░    ┌─┐
      q_1: ┤ H ├──■───░─┤ X ├──────░────┤M├
           ├───┤      ░ └─┬─┘┌───┐ ░ ┌─┐└╥┘
      q_2: ┤ H ├──────░───■──┤ H ├─░─┤M├─╫─
           └───┘      ░      └───┘ ░ └╥┘ ║ 
c_alice: 2/═══════════════════════════╩══╩═
                                      0  1 


### 3. Explain (The Concept)

*Multiple Means of Representation (Big Picture)*

Look at the circuit. We have three parts so far:
1.  **State Preparation:** Creating the cat state and the Bell pair.
2.  **Alice's Gates:** The $CNOT$ and $H$ that constitute the Bell Basis Measurement.
3.  **Alice's Measurement:** Alice measures her two qubits and stores the result in two classical bits. 

At this exact moment, the protocol is *paused*. Alice has her two bits (e.g., "10"). Bob's qubit ($q_0$) has collapsed to one of the four states from your worksheet, but *he doesn't know which one*. 

### 4. Elaborate (Apply)

*Multiple Means of Action & Expression (Higher-level skills)*

Let's run this circuit 100 times. We should see Alice get all four outcomes ("00", "01", "10", "11") with roughly equal probability (25% each).

In [ ]:
# Run the simulation
compiled_circuit = transpile(qc_alice_ops, simulator)
job = simulator.run(compiled_circuit, shots=100)
result = job.result()
counts = result.get_counts()

print("Alice's measurement outcomes:")
print(counts)
# Note: Qiskit reverses bit order. '01' means c_alice[0]=1, c_alice[1]=0

Alice's measurement outcomes:
{'00': 23, '11': 33, '10': 25, '01': 19}


### 5. Evaluate (Challenge)

*Multiple Means of Engagement (Mastery)*

This confirms the math in your notes (Section 5): "Alice...will measure 00, 01, 10, or 11, each with a 25% probability." 

**Challenge:** 

The circuit for Alice's Bell Basis Measurement ($CNOT$ then $H$) is the *exact inverse* of the circuit to *create* a Bell state ($H$ then $CNOT$). Why do you think this is the case?

In [ ]:
# Your Answer Here ---

--- 

## Task 3: Bob's Conditional Corrections & Full Protocol

### 1. Engage (Why?)

*Multiple Means of Engagement (Relevance)*

This is the final step! Alice sends her two classical bits to Bob. Bob now uses those bits to apply the *conditional* correction gates needed to retrieve the original state $|\psi
\rangle$. This is the "classical communication" part of the protocol.

### 2. Explore (Play)

*Multiple Means of Action & Expression (Fluency)*

In Qiskit, we can make a quantum gate *conditional* on the value of a classical bit. The syntax is:
`qc.x(qubit_to_apply_to).c_if(classical_bit, value)`

- If Alice measures `01` ($c_C=0, c_A=1$): Bob applies $X$.
- If Alice measures `10` ($c_C=1, c_A=0$): Bob applies $Z$.
- If Alice measures `11` ($c_C=1, c_A=1$): Bob applies $Z$ and $X$.

A simpler way to write this is:
1.  If $c_A = 1$, apply $X$ to Bob's qubit.
2.  If $c_C = 1$, apply $Z$ to Bob's qubit.

### 3. Explain (The Concept)

*Multiple Means of Representation (Big Picture)*

Let's re-examine the logic. Alice's classical bits are `cr_c` (store $q_C$ result) and `cr_a` (store $q_A$ result). Bob's qubit is $q_0$. 

The two rules are:
1.  `qc.x(qr[0]).c_if(cr_a, 1)`  # Apply X if cr_a is 1
2.  `qc.z(qr[0]).c_if(cr_c, 1)`  # Apply Z if cr_c is 1

Let's trace this:
- Alice gets "00" (cr_c=0, cr_a=0): `qc.x(qr[0]).c_if(cr_a, 1)` is false. `qc.z(qr[0]).c_if(cr_c, 1)` is false. Bob applies **I** (does nothing). (Correct)
- Alice gets "01" (cr_c=0, cr_a=1): `qc.x(qr[0]).c_if(cr_a, 1)` is true. `qc.z(qr[0]).c_if(cr_c, 1)` is false. Bob applies **X**. (Correct)
- Alice gets "10" (cr_c=1, cr_a=0): `qc.x(qr[0]).c_if(cr_a, 1)` is false. `qc.z(qr[0]).c_if(cr_c, 1)` is true. Bob applies **Z**. (Correct)
- Alice gets "11" (cr_c=1, cr_a=1): `qc.x(qr[0]).c_if(cr_a, 1)` is true. `qc.z(qr[0]).c_if(cr_c, 1)` is true. Bob applies **X** and **Z**. (Correct)

### 4. Elaborate (Apply)

*Multiple Means of Action & Expression (Higher-level skills)*

Let's build the *full* circuit. We will teleport the state $|1
\rangle$. To do this, we'll apply an $X$ gate to $q_2$ at the very beginning. 
At the very end, we'll measure Bob's qubit ($q_0$) to prove it successfully received the $|1
\rangle$ state.

In [4]:
# --- Your Code Here: Full Teleportation Circuit --- 

# Registers: 3 Qubits, 3 Classical Bits
qr = QuantumRegister(3, name="q")
cr_c = ClassicalRegister(1, name="c_c") # Alice's C-bit
cr_a = ClassicalRegister(1, name="c_a") # Alice's A-bit
cr_b = ClassicalRegister(1, name="c_b") # Bob's final bit
qc_teleport = QuantumCircuit(qr, cr_c, cr_a, cr_b)

# --- 1. State Preparation --- 
# We want to teleport the |1> state
qc_teleport.x(qr[2]) # Prepare |1> on q2 (Cat)
qc_teleport.barrier()

# Prepare |Phi+> on q1 (Alice) and q0 (Bob)
qc_teleport.h(qr[1])
qc_teleport.cx(qr[1], qr[0])
qc_teleport.barrier()

# --- 2. Alice's Bell Basis Measurement --- 
qc_teleport.cx(qr[2], qr[1]) # CNOT(C, A)
qc_teleport.h(qr[2])      # H(C)
qc_teleport.barrier()

# --- 3. Alice's Measurement --- 
qc_teleport.measure(qr[2], cr_c) # Measure q2 into cr_c
qc_teleport.measure(qr[1], cr_a) # Measure q1 into cr_a
qc_teleport.barrier()

# --- 4. Bob's Conditional Corrections --- 
# Apply X to q0 if c_c == 1
qc_teleport.x(qr[0]).c_if(cr_a, 1)
# Apply Z to q0 if c_a == 1
qc_teleport.z(qr[0]).c_if(cr_c, 1)

# --- 5. Bob's Final Measurement --- 
qc_teleport.measure(qr[0], cr_b)

print("Full Teleportation Circuit (Teleporting |1>):")
print(qc_teleport.draw())

Full Teleportation Circuit (Teleporting |1>):
             ░      ┌───┐ ░            ░        ░  ┌───┐  ┌───┐ ┌─┐
  q_0: ──────░──────┤ X ├─░────────────░────────░──┤ X ├──┤ Z ├─┤M├
             ░ ┌───┐└─┬─┘ ░ ┌───┐      ░    ┌─┐ ░  └─╥─┘  └─╥─┘ └╥┘
  q_1: ──────░─┤ H ├──■───░─┤ X ├──────░────┤M├─░────╫──────╫────╫─
       ┌───┐ ░ └───┘      ░ └─┬─┘┌───┐ ░ ┌─┐└╥┘ ░    ║      ║    ║ 
  q_2: ┤ X ├─░────────────░───■──┤ H ├─░─┤M├─╫──░────╫──────╫────╫─
       └───┘ ░            ░      └───┘ ░ └╥┘ ║  ░    ║   ┌──╨──┐ ║ 
c_c: 1/═══════════════════════════════════╩══╬═══════╬═══╡ 0x1 ╞═╬═
                                          0  ║    ┌──╨──┐└─────┘ ║ 
c_a: 1/══════════════════════════════════════╩════╡ 0x1 ╞════════╬═
                                             0    └─────┘        ║ 
c_b: 1/══════════════════════════════════════════════════════════╩═
                                                                 0 


### 5. Evaluate (The Final Test)

*Multiple Means of Engagement (Mastery)*

Now we run the final circuit. We teleported the state $|1
\rangle$. Therefore, Bob's measurement at the end should be '1' *every single time*, regardless of what Alice measured.

The output `counts` string is `c_b c_a c_c`. We only care about the first bit, `c_b` (Bob's result). It should *always* be 1.

In [6]:
# --- Run the Final Simulation --- 

compiled_teleport = transpile(qc_teleport, simulator)
job = simulator.run(compiled_teleport, shots=1024)
result = job.result()
counts = result.get_counts()

print(f"Final Measurement Results: {counts}")

# --- Verify the result --- 
print("\n--- Verification ---")
print("(Counts are 'c_b c_a c_c')")
success = True
for result_str, count in counts.items():
    bobs_bit = result_str[0] # Get the first bit
    if bobs_bit == '0':
        success = False
        print(f"FAILURE: Got result {result_str} (Bob's bit was 0)")

if success:
    print("SUCCESS! Bob's bit was 1 in all 1024 shots.")
    print("This proves the |1> state was successfully teleported.")

Final Measurement Results: {'1 0 0': 266, '1 1 1': 248, '1 1 0': 258, '1 0 1': 252}

--- Verification ---
(Counts are 'c_b c_a c_c')
SUCCESS! Bob's bit was 1 in all 1024 shots.
This proves the |1> state was successfully teleported.


**Final Challenge:** Go back to the circuit in Task 3.4. Instead of `qc_teleport.x(qr[0])` to prepare $|1
\rangle$, change it to `qc_teleport.h(qr[0])` to prepare $|+
\rangle$. 
Run the final simulation again. What do you expect Bob's final measurement `c_b` to be? Why are the `counts` now split 50/50 (e.g., `{'100': 512, '000': 512}`)? (Think about what it means to measure $|+
\rangle$ in the Z-basis).